In [ ]:
from time import time
import pandas as pd
import json
from tqdm import tqdm
import numpy as np
from gensim.models import KeyedVectors
import os, copy, pickle, string, re, nltk
from scipy.stats import truncnorm
from collections import Counter
from sklearn.model_selection import train_test_split
from itertools import product
from tensorflow.python.keras.utils import losses_utils
from sklearn.utils import class_weight
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Dense, Dropout
from gensim.models import word2vec, Phrases
from sklearn.metrics import classification_report
import argparse

In [ ]:
nltk.download('punkt')

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--data", required=True, help="path of the directory containing data files")
parser.add_argument("--target", required=True)

args = vars(parser.parse_args())
data_dir = args["data"]
target_ = args["target"]

In [ ]:
with open(data_dir + '/train', 'rb') as F:
    train = pickle.load(F)

with open(data_dir + '/test', 'rb') as F:
    test = pickle.load(F)

TweetInfoDF = pd.read_csv(data_dir + '/TweetInfoDF.csv')
TweetInfoDFText = list(TweetInfoDF['text'])
num_classes = len(np.unique(train['Tags']))

In [ ]:
def remove_punctuations_and_numbers(String):
    L = []
    for s in tokenize(String):
        if(s not in string.punctuation)and not(s>="0" and s<="9")  and not(s=="…") and 'a'<=s and s<='z':
            L.append(s)
    return " ".join(L)

def tokenize(String):
    return nltk.word_tokenize(String)

def remove_stopwords(List):
    L = []
    for s in List:
        if(s not in stopwords.words("english") and (s not in hindi_stp) and (s not in hinglish_stp)):
            L.append(s)
    return L

def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

def strip_all_entities(text):
    entity_prefixes = ['@']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)


def encoding(max_length, TAG_TWEET, TAG):
    text_tokenizer = Tokenizer()

    text_tokenizer.fit_on_texts(TAG_TWEET)
    INDEXES = text_tokenizer.word_index
    
    ENCODED_TAG_TWEET = []
    
    for xyz in TAG_TWEET:
        tok = tokenize(xyz)
        LI = []
        for _ in tok:
            LI.append(INDEXES[_])
        ENCODED_TAG_TWEET.append(LI)
    print(len(ENCODED_TAG_TWEET))

    ENCODED_TAG_TWEET=pad_sequences(ENCODED_TAG_TWEET,maxlen=max_length,padding='post',value=0.0)

    return (text_tokenizer,pd.DataFrame(list(zip(TAG, ENCODED_TAG_TWEET)), columns =['Tag', 'Tweet']))

def generate_sample_weights(training_data, class_weight_dictionary): 
    sample_weights = [class_weight_dictionary[np.where(one_hot_row==1)[0][0]] for one_hot_row in training_data]
    return np.asarray(sample_weights)

In [ ]:
cleaned_tweets = []
L = 0

VOCAB = []

for i,j in train.iterrows():
    tw = TweetInfoDFText[i].lower()
    tw = strip_links(strip_all_entities(tw))
    tw = remove_punctuations_and_numbers(tw)
    cleaned_tweets.append(tw)
    L_ = len(tokenize(tw))
    VOCAB.extend(tokenize(tw))
    if L_>L:
        L=L_

train_cleaned_tweets = copy.deepcopy(cleaned_tweets)

cleaned_tweets = []

for i,j in test.iterrows():
    tw = TweetInfoDFText[i].lower()
    tw = strip_links(strip_all_entities(tw))
    tw = remove_punctuations_and_numbers(tw)
    cleaned_tweets.append(tw)
    L_ = len(tokenize(tw))
    if L_>L:
        L=L_

test_cleaned_tweets = copy.deepcopy(cleaned_tweets)

cleaned_tweets = copy.deepcopy(train_cleaned_tweets)
cleaned_tweets.extend(test_cleaned_tweets)
TagList = pd.concat([train['Tag'], test['Tag']])
text_tokenizer, encodedtexttag = encoding(L, cleaned_tweets, TagList)

In [ ]:
X_train = copy.deepcopy(train)
X_test = copy.deepcopy(test)  
y_train = copy.deepcopy(train['Tag'])
y_test = copy.deepcopy(test['Tag'])
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)

y_train = mapping(y_train)
y_train = np.asarray(y_train)
y_train = y_train.reshape(y_train.shape[0], y_train.shape[2])
print(y_train.shape)

y_test = mapping(y_test)
y_test = np.asarray(y_test)
y_test = y_test.reshape(y_test.shape[0], y_test.shape[2])

sample_weights_train = generate_sample_weights(y_train, class_weights)

INDEXES = text_tokenizer.word_index
tok = tokenize(target_)
LI = []
for _ in tok:
    LI.append(INDEXES[_])
target_seq = pad_sequences([LI],maxlen=L,padding='post',value=0.0)
target_seq = target_seq.reshape(target_seq.shape[1],)

X_train['cleaned_tweet'] = train_cleaned_tweets
X_test['cleaned_tweet'] = test_cleaned_tweets
X_train['encoded_cleaned_tweet'] = list(encodedtexttag['Tweet'][:len(train_cleaned_tweets)])
X_test['encoded_cleaned_tweet'] = list(encodedtexttag['Tweet'][len(train_cleaned_tweets):])
X_train['target'] = [target_seq]*len(train_cleaned_tweets)
X_test['target'] = [target_seq]*len(test_cleaned_tweets)

del X_train['Text']
del X_train['Hashtag']
del X_test['Text']
del X_test['Hashtag']

In [ ]:
class BLSTM_encoder (tf.keras.layers.Layer):
  def __init__(self, hidden_dim, **kwargs):
    super(BLSTM_encoder, self).__init__()
    self.encoder = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(hidden_dim, return_sequences=True)
    )
  def call(self, x):
    return self.encoder(x)

In [ ]:
class Target_attention (tf.keras.layers.Layer):
  def __init__(self):
    super(Target_attention, self).__init__()
    self.attention_ff = tf.keras.layers.Dense(1)
    self.concat = tf.keras.layers.Concatenate(axis=-1)
  def call(self, word_input, target_input):
    target_input = tf.expand_dims(target_input, 1)
    target_input = tf.repeat(target_input, repeats=word_input.shape[1], axis=1)
    aug_input = self.concat([word_input, target_input])
    att_value = self.attention_ff(aug_input)
    att_value = tf.squeeze(att_value, [-1])
    return tf.expand_dims(tf.nn.softmax(att_value), axis=-1)

In [ ]:
class Stance_output (tf.keras.layers.Layer):
  def __init__(self, num_classes=7, **kwargs):
    super(Stance_output, self).__init__()
    self.out_ff = tf.keras.layers.Dense(num_classes, activation='softmax')

  def call(self, lstm_out, att_score):
    pre_ff = tf.reduce_mean(lstm_out * att_score, axis=-2)
    return self.out_ff(pre_ff)

In [ ]:
class TAN_model (tf.keras.models.Model):
  def __init__(self,
               embedding_dim=128,
               vocab_size=10000,
               hidden_dim=64,
               num_classes=7,
               drop_rate=0.2,
               **kwargs):
    super(TAN_model, self).__init__()
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.encoder = BLSTM_encoder(hidden_dim)
    self.attention = Target_attention()
    self.stance_output = Stance_output(num_classes=num_classes)
    self.dropout1 = tf.keras.layers.Dropout(drop_rate)
    self.dropout2 = tf.keras.layers.Dropout(drop_rate)

  def call(self, tweet, target, training):
    tweet_emb = self.embedding(tweet)
    target_emb = self.embedding(target)
    target_emb = tf.reshape(target_emb,
                            [target_emb.shape[0], target_emb.shape[1]*target_emb.shape[2]])
    tweet_enc = self.encoder(tweet_emb)
    tweet_enc = self.dropout1(tweet_enc)
    att_score = self.attention(tweet_emb, target_emb)
    att_score = self.dropout2(att_score)
    outputs = self.stance_output(tweet_enc, att_score)
    return outputs

In [ ]:
def train_step(tweet_list, actual_v, target, sample_weights_list):
    tweet_list = tf.convert_to_tensor(tweet_list, dtype=tf.int32)
    target = tf.convert_to_tensor(target, dtype=tf.int32)

    with tf.GradientTape(persistent=True) as tape:
        prediction = tan_model(tf.convert_to_tensor(tweet_list), target)
        loss = cce(prediction, actual_v, sample_weight=sample_weights_list)
        epoch_accuracy.update_state(actual_v, prediction)
        
    grads = tape.gradient(loss, tan_model.trainable_variables)
    grads = [grad if grad is not None else tf.zeros_like(var) for var, grad in zip(
        tan_model.trainable_variables, grads)]
    optimizer.apply_gradients(zip(grads, tan_model.trainable_variables))

    train_loss.update_state(loss)

In [ ]:
tan_model = TAN_model(vocab_size=len(VOCAB), num_classes=num_classes, embedding_dim=256)
learning_rate = 1e-2
optimizer=keras.optimizers.Adam(learning_rate)
cce = tf.keras.losses.CategoricalCrossentropy(reduction=tf.keras.losses.Reduction.AUTO)
epoch_accuracy = tf.keras.metrics.CategoricalAccuracy()
train_loss = tf.keras.metrics.Mean(name='train_loss')

In [ ]:
batches = []
batch_size = 32

st = 0

while(st<X_train.shape[0]):
    if st+batch_size <X_train.shape[0]:
        batches.append([X_train[st:st+batch_size]['encoded_cleaned_tweet'],
                        y_train[st:st+batch_size],
                        X_train[st:st+batch_size]['target'],
                        sample_weights_train[st:st+batch_size]])
    else:
        batches.append([X_train[st:]['encoded_cleaned_tweet'],y_train[st:],
                        X_train[st:]['target'],
                        sample_weights_train[st:]])

    st = st+batch_size

In [ ]:
n_epoch = 10

pbar = tf.keras.utils.Progbar(target=n_epoch*len(batches), width=15, interval=0.005,
                              stateful_metrics=['train_loss', 'accuracy'])
for epoch in range(0,n_epoch):
    
    for encoded_tweet_list, tag, target, sample_weights_list in batches:
          tag_ = np.array(tag)
          train_step(list(encoded_tweet_list), tag_, list(target), sample_weights_list)
          pbar.add(1, values=[("train_loss", train_loss.result()),
                              ("accuracy", epoch_accuracy.result())])
    train_loss.reset_states()
    epoch_accuracy.reset_states()

In [ ]:
Tar = [target_seq for _ in range(len(X_test))]
Tar = tf.convert_to_tensor(Tar)

val_batches = []

st = 0
while(st<X_test.shape[0]):
    if st+batch_size <X_test.shape[0]:
        val_batches.append([X_test[st:st+batch_size]['encoded_cleaned_tweet'],
                            X_test[st:st+batch_size]['target']])
    else:
        val_batches.append([X_test[st:]['encoded_cleaned_tweet'],
                            X_test[st:]['target']])

    st = st+batch_size

pred_y = []

for encoded_tweet_list, tag in val_batches:
    p_y = tan_model(tf.convert_to_tensor(list(encoded_tweet_list)),
                    tf.convert_to_tensor(list(tag)))
    pred_y.extend(p_y)

print(classification_report(np.argmax(y_test, axis=1), np.argmax(pred_y, axis=1)))